In [1]:
from anchor import utils
from anchor import anchor_tabular
import pandas as pd
import sklearn.ensemble
import numpy as np
from anchor import anchor_tabular
import os

In [2]:
feature_names = None
features_to_use = [2, 4, 5, 6, 7, 9]
categorical_features = [2, 4, 7]

target_idx = 1
dataset_folder = train_path = "../../../datasets/titanic/"
discretize = True
balance = True

def filter_fn(data):
  df = pd.DataFrame(data)

  df.dropna(inplace=True)
  ###
  return df.values.astype('|S128')

dataset = utils.load_csv_dataset(
            os.path.join(dataset_folder, 'train.csv'), target_idx, ',',
            feature_names=feature_names, features_to_use=features_to_use,
            categorical_features=categorical_features, discretize=discretize,
            balance=balance, filter_fn = filter_fn, skip_first=True)

In [3]:
import sys
sys.path.append(sys.path[0] + '/../..')

import arg_explainer as ae
import dataset_manager
import testbench
import utils
from importlib import reload
reload(ae)
reload(dataset_manager)
reload(testbench)
reload(utils)
import arg_explainer as ae
import dataset_manager
import testbench
import utils

nb_rows = 100
experiment_name = 'titanic'
dm = dataset_manager.DatasetManager(dataset, classifier=None, nb_rows=nb_rows)
print('Space size =', dm.space_size())
explainer = ae.ArgTabularExplainer(dm, experiment_name + '_' + str(nb_rows), compute=True, output_path='../../saves')
tb = testbench.Testbench(dm, exp_name=experiment_name)

# tb.explore_full_dataset(16)
# tb.explore_neighborhoods(16)

Train 0.93
Test 0.7391304347826086
Space size = 2016
titanic_100
Generating arguments
len  1 : 1 ,  0
20 potential arg checked ( 0 not minimal)
len  2 : 4 ,  30
138 potential arg checked ( 5 not minimal)
len  3 : 41 ,  34
251 potential arg checked ( 172 not minimal)
len  4 : 9 ,  2
113 potential arg checked ( 418 not minimal)
len  5 : 0 ,  0
20 potential arg checked ( 268 not minimal)
len  6 : 0 ,  0
0 potential arg checked ( 58 not minimal)
Total number of arguments:  121
Saving to  ../../saves\titanic_100_minimals.df


## Network experiments

In [4]:
G = explainer.build_attack_graph(compute=True, display_graph=False)
print('edges per node:', np.mean([len(G.edges(n)) for n in G.nodes()]))
print('nodes:', len(G.nodes()), 'attacks:', len(G.edges()))

explainer.export_graph('asp', '../../saves')

edges per node: 15.322314049586776
nodes: 121 attacks: 927


### NetworkX
<br />edges per node: 15.229508196721312
<br />nodes: 122 attacks: 929
<br />Nx graph size: 8026170
<br />Time: 93.04187512397766

In [5]:
import networkx as nx
from time import time

start = time()
print('Nx # cliques:', len([n for n in nx.find_cliques(nx.complement(G))]))
print('Time:', time() - start)

KeyboardInterrupt: 

### NetworKit

In [ ]:
import networkit as nk
from time import time

G_nk = nk.nxadapter.nx2nk(nx.complement(G))

start = time()
mc = nk.clique.MaximalCliques(G_nk)
mc.run()
print('Nk # cliques:', len([n for n in mc.getCliques()]))
# print('Nk graph size:', len([n for n in mc.run()]))
print('Time:', time() - start)


Nk # cliques: 8026170
Time: 43.98240399360657


### Graph-tool

In [6]:
import graphtools as gt
from time import time

G_gt = gt.Graph(G)
start = time()
print('Gt # cliques:', len([n for n in gt.max_cliques(G_gt)]))
print('Time:', time() - start)

ModuleNotFoundError: No module named 'graphtools'

### Aspartix - Clingo
<br />Models       : 8026170
<br />Calls        : 1
<br />Time         : 162.127s (Solving: 162.12s 1st Model: 0.00s Unsat: 0.00s)
<br />CPU Time     : 132.630s